# Table of Contents
1. [start](#start)
2. [vars](#vars)

## Start provisioning <a name="start"></a>

In [1]:
import time, os, random, string, sys, requests, urllib3, ipaddress
urllib3.disable_warnings(category=urllib3.exceptions.InsecureRequestWarning)

def prov_directory(prov):
 # create working directory
 command=f"mkdir -p {prov}"
 runcmd(command)
 os.chdir(prov)

def runcmd(cmd):
    command=str(cmd).replace('\n','').replace('\r','')
    print(command) # quite mode? just comment this out.
    output=!{command}
    return(output)

def provision(hostname, full_hostname, domainname, url, os_variant, uid, size, ram):
 # begin provisioning
 directory="/share/vms/disks/"+uid+"/"
 prov_directory(directory)
 image="provisionme.iso"
 open(str(directory+image), 'wb').write(requests.get(url, verify=False, headers={"user-agent": "Provision me harder"}, allow_redirects=True).content)
 command="qemu-img create -b "+directory+image+" -f qcow2 -F qcow2 "+directory+full_hostname+".img "+size
 runcmd(command)
 command="""echo "instance-id: """+uid+""" \n local-hostname: """+full_hostname+""" " > meta-data"""
 runcmd(command)
 config="""#cloud-config
hostname: {uid}.{hostname}
create_hostname_file: true
fqdn: {full_hostname}
prefer_fqdn_over_hostname: true
package_update: true
package_upgrade: true
users:
  - name: ansible
    ssh_authorized_keys:
        - ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIC9+2bjed1kcBrQgCbgAYNaYxb+P7v0uZDMSBvnqvTZa
    sudo: ["ALL=(ALL) NOPASSWD:ALL"]
    groups: sudo
    shell: /bin/bash
packages:
  - ncat
  - curl
  - git
  - ansible
  - wget
  - nmap
  - python3-pip
  - podman
  - qemu-system
  - isc-dhcp-client
  - snapd
  - flatpak
  - net-tools
""".format(uid=uid, hostname=hostname, full_hostname=full_hostname)
 open('user-data','w').write(config)
 command="genisoimage -output cidata.iso -V cidata -r -J user-data meta-data"
 runcmd(command)
 command="virt-install --virt-type qemu --name "+full_hostname+" --ram "+ram+" --disk "+directory+full_hostname+".img,format=qcow2 --network network=default --graphics vnc,listen=192.168.56.104 --noautoconsole --os-variant="+os_variant+" --cdrom="+directory+"cidata.iso --check all=off"
 runcmd(command)
 cleanup(hostname, full_hostname, domainname, url, os_variant, uid, size, ram)

def runWithAnsible(command, ipaddr):
    # export part to prevent timeout while waiting for commands, mostly because relying on internet access sucks.
    runcmd("""export ANSIBLE_PERSISTENT_COMMAND_TIMEOUT=600; ansible all -i """+ipaddr+""", -a '"""+command+"""' -u ansible""")

def cleanup(hostname, full_hostname, domainname, url, os_variant, uid, size, ram):
 time.sleep(120) #let this get into the os before pushing shit
 command="echo "+full_hostname
 print(runcmd(command)) # this was really just a test to see if it would run right, but it makes it easy to copy whats going on. 
 command="virsh net-dhcp-leases default |grep -i "+hostname+"|awk '{print $5}'|awk -F '/' '{print $1}'"
 ipaddr=str(runcmd(command))
 print(type(ipaddr))
 print(ipaddr)
 ipaddr=ipaddr[2:][:-2]
 print(ipaddr)
 try:
     ipaddress.ip_address(ipaddr)
 except:
     print("Something went wrong, can't find ip address")
     runcmd("virsh destroy "+full_hostname)
     runcmd("virsh undefine "+full_hostname+" --remove-all-storage")
     sys.exit(1)
 command="ssh-keyscan -H "+ipaddr+" >> ~/.ssh/known_hosts"
 time.sleep(120)
 runcmd(command)
 # I should really migrate this over to my git repo as a playbook to grab and run locally, then simply run a command to initiate that, but until then, here's some adhoc commands. 
 # this has proven to be a surprisingly stable way to run these though.
 for cmd in [
     "curl -fsSL https://get.docker.com -o /tmp/installdocker.sh",
     "curl https://get.golang.org/linux -o /tmp/installgo.sh",
     "sudo bash /tmp/installdocker.sh",
     "sudo bash /tmp/installgo.sh",
     "service docker start; service containerd start",
     "sudo service snapd start",
     "sudo snap install --classic kubectl",
     "sudo snap install --classic kubeadm", 
     "sudo snap install --classic kubelet",
     "sudo kubeadmin init",
     "sudo pip install --upgrade jupyter-lab --break-system-packages --root-user-action=ignore",
     "sudo cd /; sudo git clone https://github.com/ferasdour/automation_tools"
 ]:
     print(runWithAnsible(cmd, ipaddr))


## Set variables <a name="vars"></a>

In [2]:

# having some conflicts putting these into a function and it still being used with 
hostname="ubuntu"
domainname="prov"
domainname=domainname.split('.')[0] #primary part of domain
f=filter(str.isalpha,hostname) #leave only alpha chars in hostname
hostname="".join(f)
f=filter(str.isalpha,domainname) #Leave only alpha chars in domain name
domainname="".join(f)
characters = string.ascii_lowercase + string.digitsnt(runWithAnsible(cmd, ipaddr))

uid=''.join(random.sample(characters, 10))
full_hostname=str(uid+"."+hostname+"."+domainname+".private") #set as .private | add uid (this is just to avoid having to chcek for collisions, bad code practice here
url="http://127.0.0.1:7070/plucky-server-cloudimg-amd64.img"
os_variant="ubuntu-stable-latest"
size="80G"
ram="6144"
# i have to stop wasting network bandwidth downloading this over and over when testing, so I'm pointing to local system copy of it.
runcmd("tmux new-session -d 'cd /share/vms/images/; python3 -m http.server -b 127.0.0.1 7070'")
provision(hostname, full_hostname, domainname, url, os_variant, uid, size, ram)

tmux new-session -d 'cd /share/vms/images/; python3 -m http.server -b 127.0.0.1 7070'
mkdir -p /share/vms/disks/s2u7tpybwn/
qemu-img create -b /share/vms/disks/s2u7tpybwn/provisionme.iso -f qcow2 -F qcow2 /share/vms/disks/s2u7tpybwn/s2u7tpybwn.ubuntu.prov.private.img 80G
echo "instance-id: s2u7tpybwn  local-hostname: s2u7tpybwn.ubuntu.prov.private " > meta-data
genisoimage -output cidata.iso -V cidata -r -J user-data meta-data
virt-install --virt-type qemu --name s2u7tpybwn.ubuntu.prov.private --ram 6144 --disk /share/vms/disks/s2u7tpybwn/s2u7tpybwn.ubuntu.prov.private.img,format=qcow2 --network network=default --graphics vnc,listen=192.168.56.104 --noautoconsole --os-variant=ubuntu-stable-latest --cdrom=/share/vms/disks/s2u7tpybwn/cidata.iso --check all=off
echo s2u7tpybwn.ubuntu.prov.private
virsh net-dhcp-leases default |grep -i ubuntu|awk '{print $5}'|awk -F '/' '{print $1}'
<class 'str'>
['10.42.0.247']
10.42.0.247
ssh-keyscan -H 10.42.0.247 >> ~/.ssh/known_hosts
export ANSIBLE_PE